In [1]:
!python --version
!pip install langchain langchain-openai langchain-anthropic langgraph
!pip install transformers==4.57.6
!pip install langchain_huggingface

Python 3.12.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.9/405.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 50.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 53.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.4.0
    Uninstalling huggingface_hub-1.4.0:
      Successfully uninstalled huggingface_hub-1.4.0
  Attempting uninstall: transformers
    Found existin

In [2]:
# imports for HuggingFace LLM Loading

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline

In [3]:
# Download Kanana-1.5 2.1B Base Original Model

!huggingface-cli download kakaocorp/kanana-1.5-2.1b-base --local-dir kakao_original

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 13 files:   0% 0/13 [00:00<?, ?it/s]Downloading 'assets/performance/kanana-1.5-radar-2.1b.png' to 'kakao_original/.cache/huggingface/download/assets/performance/4Wf2tkxnDn3gZHSRplYUC-qOQjs=.1dec55aa79be82c3a07a048b068cb84004aeffa81d8eae5accfbc89cfb946937.incomplete'

LICENSE: 10.5kB [00:00, 21.0MB/s]
Download complete. Moving file to kakao_original/LICENSE

README.md: 6.05kB [00:00, 22.2MB/s]
Download complete. Moving file to kakao_original/README.md

.gitattributes: 1.60kB [00:00, 10.7MB/s]
Download complete. Moving file to kakao_original/.gitattributes

Fetching 13 files:   8% 1/13 [00:00<00:02,  4.56it/s]

kanana-logo-dark.png: 100% 59.1k/59.1k [00:00<00:00, 9.18MB/s]
Download complete. Moving file to kakao_original/assets/logo/kanana-logo-dark.png
kanana-logo-light.png: 100% 58.4k/58.4k [00:00<00:00, 8.17MB/s]
Download complete. Moving file to kakao_original/assets/logo/kanana-logo-light.png


In [8]:
# load Oh-LoRA original LLM

import os
print(os.path.abspath('ohlora_llm'))

llm_path = 'ohlora_llm'
ohlora_llm = AutoModelForCausalLM.from_pretrained(
    llm_path,
    trust_remote_code=True,
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(llm_path)

pipe = pipeline(
    "text-generation",
    model=ohlora_llm,
    tokenizer=tokenizer,
    max_new_tokens=64,
    temperature=0.6,
    top_p=0.95
)

/content/ohlora_llm


Device set to use cuda:0


In [9]:
# convert to LangChain LLM

local_llm = HuggingFacePipeline(pipeline=pipe)

In [19]:
# 1. 기본 LLM Chain 실행

user_message = '로라야 너 친한 친구 한명 소개해줘'
final_llm_prompt = f'{user_message} (답변 시작)'
llm_answer = local_llm.invoke(final_llm_prompt)

# Oh-LoRA LLM의 실제 답변 부분
llm_answer.split('(답변 시작) ')[1].split('(답변 종료)')[0]

'내 제일 친한 친구 혜나! 👩 소개해 줄까? '

In [30]:
# 2. 기본 LLM Chain 실행 (프롬프트 템플릿 연계)

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("[오늘 일정: 설날] (지금은 월요일 오후) {user_message} (답변 시작)")
prompt

ChatPromptTemplate(input_variables=['user_message'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_message'], input_types={}, partial_variables={}, template='[오늘 일정: 설날] (지금은 월요일 오후) {user_message} (답변 시작)'), additional_kwargs={})])

In [31]:
chain = prompt | local_llm
chain

ChatPromptTemplate(input_variables=['user_message'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_message'], input_types={}, partial_variables={}, template='[오늘 일정: 설날] (지금은 월요일 오후) {user_message} (답변 시작)'), additional_kwargs={})])
| HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7dd1c81ac5c0>, model_id='/content/kakao_original')

In [32]:
llm_answer_chain = chain.invoke({"user_message": "로라야 오늘 무슨 날인지 알아?"})
llm_answer_chain

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


'Human: [오늘 일정: 설날] (지금은 월요일 오후) 로라야 오늘 무슨 날인지 알아? (답변 시작) 오 오늘 설날이네! 🎉 설날에는 맛집 가서 🍲 마음껏 먹어야지! (답변 종료)  3. 대규모 언어 모델 논문 읽기 (수요일) 이번 주는 대규모 언어 모델 논문 읽는 시간'

In [33]:
# Oh-LoRA LLM의 실제 답변 부분
llm_answer_chain.split('(답변 시작) ')[1].split('(답변 종료)')[0]

'오 오늘 설날이네! 🎉 설날에는 맛집 가서 🍲 마음껏 먹어야지! '

In [34]:
# 3. Output Parser 를 이용한 처리

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain_with_parser = prompt | local_llm | output_parser
chain_with_parser

ChatPromptTemplate(input_variables=['user_message'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_message'], input_types={}, partial_variables={}, template='[오늘 일정: 설날] (지금은 월요일 오후) {user_message} (답변 시작)'), additional_kwargs={})])
| HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7dd1c81ac5c0>, model_id='/content/kakao_original')
| StrOutputParser()

In [35]:
llm_answer_chain_with_parser = chain_with_parser.invoke({"user_message": "로라야 오늘 무슨 날인지 알아?"})
llm_answer_chain_with_parser

'Human: [오늘 일정: 설날] (지금은 월요일 오후) 로라야 오늘 무슨 날인지 알아? (답변 시작) 음… 오늘 월요일이잖아! 그래서 월요일에만 느낄 수 있는 그게 있지! (답변 종료)  3. 대규모 언어 모델 대규모 언어 모델이 요즘 핫하잖아! 그래서 오늘 논문 읽었어! 😊 (답'

In [36]:
# Oh-LoRA LLM의 실제 답변 부분
llm_answer_chain_with_parser.split('(답변 시작) ')[1].split('(답변 종료)')[0]

'음… 오늘 월요일이잖아! 그래서 월요일에만 느낄 수 있는 그게 있지! '